In [ ]:
# Use pip to install the fuzzywuzzy package
!pip install fuzzywuzzy

In [ ]:
# Import the pandas library for data manipulation
import pandas as pd

# Import the numpy library for numerical operations
import numpy as np

# Import the cosine_similarity function from sklearn.metrics.pairwise
from sklearn.metrics.pairwise import cosine_similarity

# Import the fuzz and process functions from fuzzywuzzy for string similarity calculations
from fuzzywuzzy import fuzz, process

In [ ]:
# Load the movie data from the 'movies.csv' file into a DataFrame named 'movies'
movies = pd.read_csv('movies.csv')

# Load the ratings data from the 'ratings.csv' file into a DataFrame named 'ratings'
ratings = pd.read_csv('ratings.csv')

In [ ]:
# Preprocess the Data

# Merge the 'ratings' and 'movies' DataFrames based on the 'movieId' column
movie_ratings = pd.merge(ratings, movies, on='movieId')

# Create a user-item matrix where rows represent users and columns represent movie titles
user_movie_matrix = movie_ratings.pivot_table(index='userId', columns='title', values='rating')

# Fill any NaN (missing) values in the user-movie matrix with 0 to represent no rating
user_movie_matrix = user_movie_matrix.fillna(0)

In [ ]:
# Compute Item Similarity

# Calculate the cosine similarity between item vectors (transpose of the user-movie matrix)
item_similarity = cosine_similarity(user_movie_matrix.T)

# Create a DataFrame to store the item similarity matrix with movie titles as row and column indices
item_similarity_df = pd.DataFrame(item_similarity, index=user_movie_matrix.columns, columns=user_movie_matrix.columns)

In [ ]:
def find_closest_movie(user_input, dataset, threshold=60):
    """
    Find the closest matching movie title in the dataset based on user input.

    Args:
        user_input (str): The user's input.
        dataset (pandas.DataFrame): The dataset containing movie titles.
        threshold (int): The minimum similarity score for a match (default is 60).

    Returns:
        str: The closest matching movie title, or None if no matching movie is found.
    """
    # Use the fuzzywuzzy process.extractOne function to find the closest matching movie title
    matches = process.extractOne(user_input, dataset['title'], scorer=fuzz.token_set_ratio)

    # Check if the similarity score of the closest match is above or equal to the threshold
    if matches[1] >= threshold:
        return matches[0]  # Return the closest matching movie title
    else:
        return None  # Return None if no matching movie is found.

In [ ]:
def recommend_movies(user_input, num_recommendations=5, threshold=60):
    """
    Recommend movies based on user input.

    Args:
        user_input (str): The user's input.
        num_recommendations (int): Number of movie recommendations to return (default is 5).
        threshold (int): The minimum similarity score for a match (default is 60).

    Returns:
        list: Recommended movie titles.
    """
    # Find the closest matching movie based on user input
    closest_movie = find_closest_movie(user_input, movies, threshold)

    if closest_movie:
        # Retrieve similarity scores for movies similar to the closest movie
        similar_scores = item_similarity_df[closest_movie]

        # Sort movies by similarity score in descending order and select the top recommendations
        similar_movies = similar_scores.sort_values(ascending=False).index[1:num_recommendations+1]

        return similar_movies.tolist()  # Convert to a list
    else:
        return []  # Return an empty list if no closest movie is found.

In [ ]:
# User input for the movie title
user_input = "Titanic"

# Generate movie recommendations based on user input
recommendations = recommend_movies(user_input)

# Check if recommendations are available
if recommendations:
    # Print recommended movies for the user's input
    print(f"Recommended movies for '{user_input}':")
    for movie in recommendations:
        print(movie)
else:
    # Print a message when no matching movie is found
    print("No matching movie found.")

Recommended movies for 'Titanic':
Men in Black (a.k.a. MIB) (1997)
Star Wars: Episode I - The Phantom Menace (1999)
Saving Private Ryan (1998)
Shrek (2001)
Catch Me If You Can (2002)
